In [1]:
import os
import json
import pandas as pd
import numpy as np
from pytorch_transformers import BertModel,BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange



In [2]:
from my_process_json import *

In [20]:
data_reader = MyDataReader(data_path='/home/akobtan/NLP_Project/Project/clicr_dataset/'+ 'train1.0.json')

In [21]:
mydata = data_reader.send_batches()

In [22]:
paragraphs = [e['p'] for e in mydata]
paragraph_tags = [e['p_tags'] for e in mydata]
queries = [e['q'] for e in mydata]
query_tags = [e['q_tags'] for e in mydata]

In [49]:
#creating embeddings for a batch sample from CLICR
tags_vals = ['O','I','B']
tag2idx = {t: i for i, t in enumerate(tag_vals)}

In [24]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [25]:
MAX_LEN = 370
bs = 8
SEED = 520

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [27]:
device

device(type='cuda')

In [28]:
torch.cuda.get_device_name(0) 

'Tesla V100-SXM2-16GB'

In [29]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
"""sentences = []
for i,_ in enumerate(paragraphs):
    sentences.append(paragraphs[i]+ ' <sep> ' + queries[i])
assert len(sentences)==len(paragraphs)"""

"sentences = []\nfor i,_ in enumerate(paragraphs):\n    sentences.append(paragraphs[i]+ ' <sep> ' + queries[i])\nassert len(sentences)==len(paragraphs)"

In [30]:
split_size = 150
tokenized_texts = []
for i,paragraph in enumerate(paragraphs):
    #number of splits for current paragraph
    n_splits = int(len(paragraph.split())/split_size)
    #tokenize split paragraph text appended by full query at the end with <sep> token between paragraph and query
    tokenized_texts += [tokenizer.tokenize(s) for s in [' '.join(paragraph.split()[(split_size*split):(split_size*(split+1))]) + ' <sep> ' + queries[i] for split in range(n_splits)]]
    #tokenize remainder of splits
    if int(len(paragraph.split()) % split_size) > 0:
        tokenized_texts += [tokenizer.tokenize(' '.join(paragraph.split()[(n_splits*split_size):]) + ' <sep> ' + queries[i])]
        
        

In [31]:
print(tokenized_texts[0])

['isolated', 'cr', '##anial', 'distortion', 'mimic', '##king', 'cap', '##ut', 'su', '##cc', '##ede', '##num', 'from', 'am', '##nio', '##tic', 'band', 'disruption', 'without', 'any', 'neurological', 'abnormal', '##ity', 'summary', 'this', 'report', 'describes', 'a', 'term', 'newborn', 'with', 'isolated', 'distortion', 'in', 'the', 'left', 'par', '##ie', '##tal', 'bone', 'without', 'any', 'other', 'visible', 'congenital', 'anomaly', ',', 'due', 'to', 'am', '##nio', '##tic', 'band', 'disruption', '.', 'a', 'skull', 'x', '-', 'ray', ',', 'ultrasound', 'scan', 'and', 'subsequent', 'mri', 'scan', 'of', 'the', 'brain', 'did', 'not', 'show', 'any', 'apparent', 'distortion', 'apart', 'from', 'depression', 'and', 'con', '##ca', '##vity', 'in', 'the', 'left', 'par', '##ie', '##tal', 'bone', '.', 'the', 'purpose', 'of', 'this', 'case', 'report', 'is', 'to', 'raise', 'awareness', 'of', 'this', 'possible', ',', 'mild', 'outcome', 'of', 'this', 'little', '-', 'known', 'entity', ',', 'which', 'may', '

In [32]:
len(tokenized_texts)

4954

In [33]:
max([len(t) for t in tokenized_texts])

323

In [34]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([ 7275, 13675, 27532, 20870, 23150,  6834,  6178,  4904, 10514,
        9468, 14728, 19172,  2013,  2572, 27678,  4588,  2316, 20461,
        2302,  2151, 23130, 19470,  3012, 12654,  2023,  3189,  5577,
        1037,  2744, 20662,  2007,  7275, 20870,  1999,  1996,  2187,
       11968,  2666,  9080,  5923,  2302,  2151,  2060,  5710, 27480,
       28685,  1010,  2349,  2000,  2572, 27678,  4588,  2316, 20461,
        1012,  1037,  7412,  1060,  1011,  4097,  1010, 27312, 13594,
        1998,  4745, 27011, 13594,  1997,  1996,  4167,  2106,  2025,
        2265,  2151,  6835, 20870,  4237,  2013,  6245,  1998,  9530,
        3540, 24872,  1999,  1996,  2187, 11968,  2666,  9080,  5923,
        1012,  1996,  3800,  1997,  2023,  2553,  3189,  2003,  2000,
        5333,  7073,  1997,  2023,  2825,  1010, 10256,  9560,  1997,
        2023,  2210,  1011,  2124,  9178,  1010,  2029,  2089, 23150,
        6178,  4904, 10514,  9468, 11960,  2638,  2819,  1006,  9587,
       21285,  2075,

In [35]:
len(input_ids)

4954

In [36]:
labels = []
for i,paragraph_tag in enumerate(paragraph_tags):
    #number of splits for current paragraph tags
    n_splits = int(len(paragraph_tag)/split_size)
    labels+=[paragraph_tag[(split_size*split):(split_size*(split+1))]+ ['O'] + query_tags[i] for split in range(n_splits)]
    #tokenize remainder of splits
    if int(len(paragraph_tag) % split_size) > 0:
        labels+=[paragraph_tag[(n_splits*split_size):]+ ['O'] + query_tags[i]]
    
len(labels)

4954

In [37]:
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [38]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
tags[0]

array([0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
len(tags)

4954

In [40]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [41]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=SEED, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=SEED, test_size=0.1)

In [42]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [149]:
mydata = []
paragraphs = []
paragraph_tags = []
queries = []
query_tags = []
tokenized_texts = []
input_ids = []
tags = []
labels = []
attention_masks = []
tokenizer = []

In [43]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [44]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))


In [45]:
model.cuda();

In [46]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [47]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
epochs = 200
max_grad_norm = 1.0
prev_f1 = 0
i=0
for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    c_f1 = f1_score(pred_tags, valid_tags)
    print("F1-Score: {}".format(c_f1))
    if (c_f1 - prev_f1 < 0.05) and i>5 and c_f1>0.6:
        print('Done training')
        break
    else:
        i+=1
        prev_f1 = c_f1



Epoch:   0%|          | 0/200 [00:00<?, ?it/s]

Train loss: 0.03709364050517767
Validation loss: 0.03278004621989244
Validation Accuracy: 0.9971065823888405




Epoch:   0%|          | 1/200 [02:30<8:20:34, 150.93s/it]

F1-Score: 0
Train loss: 0.03300684164854027
Validation loss: 0.03344385578267036
Validation Accuracy: 0.9971065823888405




Epoch:   1%|          | 2/200 [05:03<8:19:14, 151.29s/it]

F1-Score: 0
Train loss: 0.029093633812274717
Validation loss: 0.03406497028340856
Validation Accuracy: 0.9971065823888405




Epoch:   2%|▏         | 3/200 [07:34<8:16:58, 151.36s/it]

F1-Score: 0
Train loss: 0.025535989621584054
Validation loss: 0.0340232374522114
Validation Accuracy: 0.9971065823888405




Epoch:   2%|▏         | 4/200 [10:05<8:14:20, 151.33s/it]

F1-Score: 0


In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

In [ ]:
from pytorch_pretrained_bert import WEIGHTS_NAME, CONFIG_NAME

output_dir = "./models/"

# Step 1: Save a model, configuration and vocabulary that you have fine-tuned

# If we have a distributed model, save only the encapsulated model
# (it was wrapped in PyTorch DistributedDataParallel or DataParallel)
model_to_save = model.module if hasattr(model, 'module') else model

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(output_dir)

In [ ]:
# Step 2: Re-load the saved model and vocabulary

# Example for a Bert model
output_dir = "./models/"
model = BertForTokenClassification.from_pretrained(output_dir,num_labels=len(tag2idx))
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True)  # Add specific options if needed
model.cuda();

In [ ]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))